In [1]:
# Some basic imports

import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
import datetime as dt
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from scipy import stats
from scipy.stats import gmean
pd.options.display.max_rows = 12
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
from scipy.stats import ttest_1samp

# Q 1
What information events happen during market hours and outside of market hours? Specifically
- When are earnings of companies announced?
- Typically, are macro announcements like non-farm payrolls during trading day or before market hours? Hm, why would macro announcements matter for individual stocks?

## Answer to Q 1
- Earnings of companies are typically announced outside the market hours. The stock price usually surge before post-close earnings announcements and then drop after the announcement.
- Macro announcements like non-farm payrolls are typically announced during the trading day.
- Macro announcements matter for individual stocks because the macro announcements can affect the stock prices. Good or bad news may affect investor's sentiment and expectation in stock markets. For example, if the non-farm payrolls are better than expected, the stock prices can go up.
- For example, if the Federal Reserve raises the interest rates, the stock prices can go down. If the Federal Reserve lowers the interest rates, the stock prices can go up.
- For example, if the inflation rate is higher than expected, the stock prices can go down. If the inflation rate is lower than expected, the stock prices can go up.

# Q 2
Now read in the parquet file I provided. What is the correlation of intraday, overnight momentum and regular momentum?

In [2]:
# read the data
df = pd.read_parquet('data/hw2_mfin7037_data.parquet', engine='pyarrow')
# convert the date to datetime
df['date'] = pd.to_datetime(df['date'])
# start the data in 1993
df = df.query("date>='1993-01-01'")

# Filter out rows with missing values in the specified columns
hw2_data = df[
    (df['ret'].notnull()) &
    (df['intraday_ret_month'].notnull()) &
    (df['overnight_ret_month'].notnull()) &
    (df['mom'].notnull()) &
    (df['mom_intraday'].notnull()) &
    (df['mom_overnight'].notnull()) &
    (df['mcap_lag1'].notnull()) &
    (np.abs(df['prc_lag1']) >= 1)
]

# Ensure each permno has at least 8 observations recently
hw2_data = hw2_data.groupby('permno').filter(lambda x: len(x) >= 8)
# reset index
hw2_data.reset_index(drop=True, inplace=True)
# Sort the data by date
df = df.sort_values(['date'])

hw2_data

,permno,date,ret,intraday_ret_month,overnight_ret_month,mcap_lag1,prc_lag1,mom_intraday,mom,mom_overnight,mcap_bin
0,10094.0,1993-01-29,0.144578,-0.073377,0.235213,89650.375000,10.3750,-0.533850,-0.151231,0.382617,6.0
1,10094.0,1993-02-26,-0.031579,-0.152771,0.143045,102611.875000,11.8750,-0.854891,-0.368651,0.486240,6.0
2,10094.0,1993-03-31,-0.065217,-0.105752,0.045330,99371.500000,11.5000,-0.950865,-0.328925,0.621939,6.0
3,10094.0,1993-04-30,0.000000,0.113035,-0.101555,92912.250000,10.7500,-0.580953,-0.103184,0.477767,6.0
4,10094.0,1993-05-28,-0.069767,-0.196001,0.157008,92912.250000,10.7500,-0.824477,-0.264152,0.560324,6.0
...,...,...,...,...,...,...,...,...,...,...,...
1652474,93399.0,2018-02-28,-0.042735,0.097550,-0.127817,28442.698957,1.1700,-0.926831,-0.441683,0.482610,1.0
1652475,93399.0,2018-03-29,0.165893,0.228345,-0.050844,27250.720116,1.1200,-1.103907,-0.592295,0.509077,1.0
1652476,93399.0,2018-04-30,-0.208761,-0.122460,-0.098346,31771.418853,1.3058,-0.838037,-0.647477,0.190561,1.0
1652477,93399.0,2018-05-31,0.099206,0.284212,-0.144063,25138.789822,1.0332,-0.224259,0.041994,0.266253,1.0


In [3]:
# calculate the correlation of intraday, overnight momentum and regular momentum
correlation = hw2_data[['mom_intraday', 'mom_overnight', 'mom']].corr()

# Rename the columns
correlation.columns = ['Intraday Momentum', 'Overnight Momentum', 'Regular Momentum']
correlation.index = ['Intraday Momentum', 'Overnight Momentum', 'Regular Momentum']

# Result of the correlation is limited to 4 decimal places
correlation = correlation.round(4)

# Display the correlation
print('Correlation of Intraday, Overnight Momentum and Regular Momentum:')
correlation

Correlation of Intraday, Overnight Momentum and Regular Momentum:


,Intraday Momentum,Overnight Momentum,Regular Momentum
Intraday Momentum,1.0000,-0.6637,0.5818
Overnight Momentum,-0.6637,1.0000,0.1546
Regular Momentum,0.5818,0.1546,1.0000


# Q 3

# Q 4
Let’s now report three tables. Kick out anything that in the bottom 20% of market capitalization (already computed) at portfolio formation time (remember the return reported is the return you earn from close of t-1 to close of time t), and anything with a price less than 5 at that time. Now report the 12 row x 11 column table, where the columns are the average returns/t-statistics of those returns for portfolios 1..10 and then the long-short portfolio, and there are six sets of rows, one for EW, VW, EW overnight, VW overnight, EW intraday, EW overnight.

- Based on the tables you produced, does intraday momentum predict intraday future returns or overnight returns? What about total returns? Likewise, does overnight momentum predict intraday future returns or overnight returns? What about total returns? Interesting patterns, right?

In [4]:
# Assign binning function
def apply_quantiles(inputdata, col, bins=10):
    def quantile_bin(s):
        s = s.dropna()  # Drop missing values
        try:
            # Try pd.qcut first
            return pd.qcut(s, q=bins, labels=False, duplicates="drop") + 1
        except ValueError:
            # Fallback to ranking if pd.qcut fails (e.g., due to insufficient unique values)
            return np.ceil(s.rank(method='min') / len(s) * bins)

    # Apply the binning function within each date group
    return inputdata.groupby('date')[col].transform(quantile_bin)

In [5]:
def produce_table(portfolios, subsetting=lambda df: df):
    bins_df = portfolios['bin'][['date', 'bin', 'ew', 'vw',
                                 'ew_intraday', 'vw_intraday',
                                 'ew_overnight', 'vw_overnight']].copy()
    pnl_df = portfolios['pnl'][['date', 'ew', 'vw',
                                'ew_intraday', 'vw_intraday',
                                'ew_overnight', 'vw_overnight']].copy()
    pnl_df['bin'] = 11
    combined = pd.concat([bins_df, pnl_df], ignore_index=True)
    combined = subsetting(combined)

    return_measures = ['ew', 'vw', 'ew_intraday', 'vw_intraday', 'ew_overnight', 'vw_overnight']
    result = []

    for measure in return_measures:
        grouped = combined.groupby('bin')[measure].agg(
            mean=lambda x: round(x.mean() * 100, 3),
            t_stat=lambda x: f"({round(ttest_1samp(x.dropna(), 0).statistic, 3)})"
        ).reset_index()

        mean_row = {'Portfolio': measure.upper().replace('_', ' ')}
        t_stat_row = {'Portfolio': ''}
        for _, row in grouped.iterrows():
            bin_ = row['bin']
            mean_row[bin_] = row['mean']
            t_stat_row[bin_] = row['t_stat']

        result.append(pd.DataFrame([mean_row]))
        result.append(pd.DataFrame([t_stat_row]))

    table = pd.concat(result, ignore_index=True)
    if 11 in table.columns:
        table = table.rename(columns={11: '10-1'})

    valid_portfolios = list(range(1, 11)) + ['10-1']
    cols_to_keep = ['Portfolio'] + [col for col in table.columns if col in valid_portfolios]
    table = table[cols_to_keep]

    return table

In [6]:
# Filter out stocks in the bottom 20% of market capitalization and those with price < 5
mcap_20th_percentile = hw2_data['mcap_lag1'].quantile(0.2)
filtered_data = hw2_data.loc[
    (hw2_data['mcap_lag1'] > mcap_20th_percentile) &
    (hw2_data['prc_lag1'] >= 5)
].copy()

# Construct portfolios based on 'mom', 'mom_intraday', and 'mom_overnight'
filtered_data['bin_mom'] = apply_quantiles(filtered_data, 'mom')
filtered_data['bin_mom_intraday'] = apply_quantiles(filtered_data, 'mom_intraday')
filtered_data['bin_mom_overnight'] = apply_quantiles(filtered_data, 'mom_overnight')

In [7]:
filtered_data

,permno,date,ret,intraday_ret_month,overnight_ret_month,mcap_lag1,prc_lag1,mom_intraday,mom,mom_overnight,mcap_bin,bin_mom,bin_mom_intraday,bin_mom_overnight
0,10094.0,1993-01-29,0.144578,-0.073377,0.235213,89650.375000,10.375,-0.533850,-0.151231,0.382617,6.0,1,1,10
1,10094.0,1993-02-26,-0.031579,-0.152771,0.143045,102611.875000,11.875,-0.854891,-0.368651,0.486240,6.0,1,1,10
2,10094.0,1993-03-31,-0.065217,-0.105752,0.045330,99371.500000,11.500,-0.950865,-0.328925,0.621939,6.0,1,1,10
3,10094.0,1993-04-30,0.000000,0.113035,-0.101555,92912.250000,10.750,-0.580953,-0.103184,0.477767,6.0,2,1,10
4,10094.0,1993-05-28,-0.069767,-0.196001,0.157008,92912.250000,10.750,-0.824477,-0.264152,0.560324,6.0,1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652402,93399.0,2012-02-29,-0.087186,-0.013701,-0.074506,203746.554620,7.570,-0.474636,-0.293305,0.181675,4.0,2,1,9
1652403,93399.0,2012-03-30,-0.083574,-0.022338,-0.062793,181090.366001,6.910,-0.427135,-0.139824,0.287656,4.0,2,1,10
1652404,93399.0,2012-04-30,-0.169841,-0.133245,-0.042225,169564.505134,6.300,-0.189492,-0.031262,0.158456,4.0,1,1,7
1652405,93399.0,2012-05-31,-0.254302,-0.209769,-0.056355,140765.450513,5.230,-0.135515,-0.105887,0.029687,4.0,1,2,3


In [8]:
# Define function to calculate portfolios and PNL
def calculate_portfolios_and_pnl(data, bin_column):
    # Calculate portfolio returns
    portfolios = (
        data
        .groupby(['date', bin_column])
        .agg(
            ew=('ret', 'mean'),
            vw=('ret', lambda x: (x * data.loc[x.index, 'mcap_lag1']).sum() / data.loc[x.index, 'mcap_lag1'].sum()),
            ew_intraday=('intraday_ret_month', 'mean'),
            vw_intraday=('intraday_ret_month', lambda x: (x * data.loc[x.index, 'mcap_lag1']).sum() / data.loc[x.index, 'mcap_lag1'].sum()),
            ew_overnight=('overnight_ret_month', 'mean'),
            vw_overnight=('overnight_ret_month', lambda x: (x * data.loc[x.index, 'mcap_lag1']).sum() / data.loc[x.index, 'mcap_lag1'].sum())
        )
        .reset_index()
    )

    # Calculate PNL (long-short portfolio)
    pnl = (
        portfolios
        .groupby('date')
        .apply(lambda x: pd.Series({
            'ew': x['ew'].iloc[-1] - x['ew'].iloc[0],
            'vw': x['vw'].iloc[-1] - x['vw'].iloc[0],
            'ew_intraday': x['ew_intraday'].iloc[-1] - x['ew_intraday'].iloc[0],
            'vw_intraday': x['vw_intraday'].iloc[-1] - x['vw_intraday'].iloc[0],
            'ew_overnight': x['ew_overnight'].iloc[-1] - x['ew_overnight'].iloc[0],
            'vw_overnight': x['vw_overnight'].iloc[-1] - x['vw_overnight'].iloc[0]
        }))
        .reset_index()
    )

    return {'bin': portfolios, 'pnl': pnl}

In [9]:
# Calculate portfolios and pnl for each portfolio
portfolios_mom = calculate_portfolios_and_pnl(filtered_data, 'bin_mom')
portfolios_mom_intraday = calculate_portfolios_and_pnl(filtered_data, 'bin_mom_intraday')
portfolios_mom_overnight = calculate_portfolios_and_pnl(filtered_data, 'bin_mom_overnight')

# rename the bin columns
portfolios_mom['bin'].rename(columns={'bin_mom': 'bin'}, inplace=True)
portfolios_mom_intraday['bin'].rename(columns={'bin_mom_intraday': 'bin'}, inplace=True)
portfolios_mom_overnight['bin'].rename(columns={'bin_mom_overnight': 'bin'}, inplace=True)

In [10]:
# Produce tables
table_mom = produce_table(portfolios_mom)
table_mom_intraday = produce_table(portfolios_mom_intraday)
table_mom_overnight = produce_table(portfolios_mom_overnight)

In [11]:
# Display the table for Regular Momentum
print('Table for Regular Momentum:')
table_mom

Table for Regular Momentum:


,Portfolio,1,2,3,4,5,6,7,8,9,10,10-1
0,EW,0.105,0.694,0.843,0.961,0.947,1.037,0.962,1.058,1.049,1.375,1.269
1,,(0.238),(2.097),(2.915),(3.781),(3.895),(4.379),(4.008),(4.019),(3.552),(3.446),(3.811)
2,VW,0.176,0.839,0.7,0.845,0.928,0.797,0.808,0.849,0.722,1.4,1.224
3,,(0.399),(2.551),(2.444),(3.382),(4.083),(3.644),(3.621),(3.599),(2.627),(3.698),(3.017)
4,EW INTRADAY,-0.886,0.595,0.904,1.083,1.007,0.992,0.806,0.707,0.447,-0.039,0.848
5,,(-2.309),(2.071),(3.676),(5.005),(4.989),(5.19),(4.159),(3.362),(1.887),(-0.124),(2.758)
6,VW INTRADAY,-0.746,0.504,0.422,0.611,0.586,0.417,0.276,0.005,-0.406,-0.723,0.024
7,,(-2.004),(1.821),(1.758),(2.844),(3.026),(2.359),(1.492),(0.025),(-1.799),(-2.348),(0.065)
8,EW OVERNIGHT,1.698,0.554,0.266,0.184,0.197,0.274,0.407,0.629,0.931,1.85,0.153
9,,(8.688),(3.729),(1.984),(1.486),(1.565),(2.18),(3.16),(4.531),(5.91),(9.058),(1.05)


In [12]:
# Display the table for Intraday Momentum
print('Table for Intraday Momentum:')
table_mom_intraday

Table for Intraday Momentum:


,Portfolio,1,2,3,4,5,6,7,8,9,10,10-1
0,EW,0.099,0.738,0.944,1.044,0.989,0.999,1.022,1.045,0.966,1.184,1.084
1,,(0.218),(2.167),(3.224),(3.957),(3.964),(4.138),(4.322),(4.201),(3.579),(3.603),(4.014)
2,VW,0.443,0.668,0.882,0.975,0.898,0.892,0.816,0.839,0.937,1.006,0.563
3,,(0.989),(2.003),(3.141),(4.039),(3.932),(3.872),(3.631),(3.557),(3.676),(2.878),(1.681)
4,EW INTRADAY,-2.275,-0.3,0.293,0.592,0.668,0.769,0.944,1.049,1.212,2.663,4.939
5,,(-5.703),(-1.047),(1.181),(2.757),(3.231),(3.937),(4.898),(5.226),(5.554),(9.538),(16.751)
6,VW INTRADAY,-1.735,-0.437,0.044,0.28,0.338,0.409,0.464,0.535,0.654,0.801,2.536
7,,(-4.556),(-1.582),(0.191),(1.466),(1.83),(2.181),(2.487),(2.74),(3.093),(2.831),(8.211)
8,EW OVERNIGHT,3.252,1.437,0.922,0.651,0.494,0.388,0.257,0.204,0.073,-0.689,-3.941
9,,(14.595),(8.854),(6.487),(4.835),(3.933),(3.039),(2.043),(1.527),(0.52),(-4.055),(-21.658)


In [13]:
# Display the table for Overnight Momentum
print('Table for Overnight Momentum:')
table_mom_overnight

Table for Overnight Momentum:


,Portfolio,1,2,3,4,5,6,7,8,9,10,10-1
0,EW,0.66,0.855,0.975,0.996,1.047,0.987,1.025,1.047,0.926,0.512,-0.147
1,,(2.255),(3.086),(3.696),(3.942),(4.139),(3.938),(3.852),(3.576),(2.693),(1.203),(-0.606)
2,VW,0.704,0.526,0.907,0.96,0.789,0.789,0.925,0.908,0.797,0.916,0.212
3,,(2.212),(1.975),(3.686),(4.05),(3.612),(3.602),(3.862),(3.752),(2.602),(2.129),(0.644)
4,EW INTRADAY,3.364,1.601,1.231,1.003,0.804,0.509,0.266,-0.082,-0.691,-2.395,-5.758
5,,(12.647),(6.667),(5.515),(4.787),(3.881),(2.468),(1.23),(-0.351),(-2.468),(-6.802),(-21.75)
6,VW INTRADAY,2.388,1.262,1.146,0.954,0.491,0.273,0.025,-0.281,-1.075,-2.163,-4.551
7,,(8.478),(5.876),(5.664),(4.877),(2.761),(1.497),(0.131),(-1.425),(-4.293),(-6.337),(-14.914)
8,EW OVERNIGHT,-1.968,-0.438,-0.033,0.191,0.417,0.646,0.985,1.403,2.025,3.765,5.733
9,,(-12.32),(-3.224),(-0.253),(1.522),(3.18),(5.028),(7.222),(9.246),(11.473),(16.123),(26.55)


# Answer to Q 4
- Based on the tables produced, the intraday momentum predicts the intraday future returns more than the overnight returns. A hedge portfolio based on past one-month intraday returns earns an average EW intraday return of 4.939% per month with an associated t-statistic of 16.75. However, hedge portfolio based on past one-month intraday returns earns an average EW overnight return of -3.94% per month with an associated t-statistic of -21.658. The intraday momentum also predicts the total returns more than the overnight returns, but less than the intraday future returns.

- The overnight momentum predicts the overnight returns more than the intraday future returns. A hedge portfolio based on past one-month overnight returns earns an average EW intraday return of -5.758% per month with an associated t-statistic of -21.75. However, hedge portfolio based on past one-month overnight returns earns an average EW overnight return of 5.733% per month with an associated t-statistic of 26.55. The overnight momentum also predicts the total returns more than the intraday future returns, but less than the overnight returns.

# Extra credit to Q 4
- Extra credit (1 point): For just the regular momentum portfolios, for a typical month, what is the fraction of winners that outperform the market return (i.e. from the fama French factor file)? I would first compute the monthly fraction, then average across time. Why are portfolios necessary in our quant trading strategies, why can’t we just pick one stock?

In [14]:
# Read the Fama French factor file
ff_data = pd.read_parquet('data/hw1_factors.parquet', engine='pyarrow')
ff_data['dt'] = pd.to_datetime(ff_data['dt'])
ff_data = ff_data.sort_values(by='dt', ascending=True)
ff_data.set_index('dt', inplace=True)

# Calculate market return in decimal format
ff_data['Mkt'] = ff_data['mkt_rf'] + ff_data['rf']
ff_data['Mkt'] = ff_data['Mkt']/100
ff_data['mkt_rf'] = ff_data['mkt_rf']/100
ff_data['rf'] = ff_data['rf']/100

# Ensure the data is not missing
ff_data = ff_data.dropna()

# Convert the daily data to monthly data to align with the monthly momentum data
ff_data_mkt = ff_data.resample('M').agg({
    'Mkt': lambda x: (1 + x).prod() - 1  # Cumulative monthly return
}).reset_index()

In [15]:
ff_data_mkt

,dt,Mkt
0,1963-07-31,-0.001394
1,1963-08-31,0.053341
2,1963-09-30,-0.012966
3,1963-10-31,0.028168
4,1963-11-30,-0.005799
...,...,...
732,2024-07-31,0.016669
733,2024-08-31,0.020874
734,2024-09-30,0.021422
735,2024-10-31,-0.005719


In [24]:
# Make a copy of the filtered data
regular_mom_data = portfolios_mom['bin'].copy()

# Convert the date column to monthly end to align with the monthly Fama French factor data
regular_mom_data['date'] = regular_mom_data['date'] + MonthEnd(0)

# Sort the data by date
regular_mom_data = regular_mom_data.sort_values(['date'])

# Merge the regular momentum data with the monthly Fama French factor data
combined_data = regular_mom_data.merge(ff_data_mkt, left_on='date', right_on='dt', how='inner')

# sort the data by date
combined_data = combined_data.sort_values(['date'])

# Calculate the fraction of winners for each month
combined_data['winner'] = combined_data['ew'] > combined_data['Mkt']
monthly_fraction = combined_data.groupby(['date', 'bin'])['winner'].mean().reset_index()

monthly_fraction

,date,bin,winner
0,1993-01-31,1,1.0
1,1993-01-31,2,1.0
2,1993-01-31,3,1.0
3,1993-01-31,4,1.0
4,1993-01-31,5,1.0
...,...,...,...
3565,2022-09-30,6,1.0
3566,2022-09-30,7,1.0
3567,2022-09-30,8,1.0
3568,2022-09-30,9,1.0


In [28]:
# Average the monthly fractions across time
average_fraction = monthly_fraction.groupby(['bin'])['winner'].mean().reset_index()
average_fraction

,bin,winner
0,1,0.420168
1,2,0.467787
2,3,0.484594
3,4,0.521008
4,5,0.487395
5,6,0.532213
6,7,0.495798
7,8,0.518207
8,9,0.523810
9,10,0.563025


In [29]:
# average fraction of winners that outperform the market return across time
average = average_fraction['winner'].mean() * 100
print('Average fraction of winners that outperform the market return: {:.2f}%'.format(average))

Average fraction of winners that outperform the market return: 50.14%


In [30]:
# construct long-short portfolio
long_short = portfolios_mom['pnl'].copy()
# adjust date to align with the monthly end Fama French factor data
long_short['date'] = long_short['date'] + MonthEnd(0)

# merge the long-short portfolio with the monthly Fama French factor data
long_short = long_short.merge(ff_data_mkt, left_on='date', right_on='dt', how='inner')

# calculate the fraction of winners for each month for the long-short portfolio
long_short['winner'] = long_short['ew'] > long_short['Mkt']
long_short_monthly_fraction = long_short.groupby(['date'])['winner'].mean().reset_index()

# Average the monthly fractions across time for the long-short portfolio
long_short_average_fraction = long_short_monthly_fraction['winner'].mean() * 100
print('Average fraction of winners that outperform the market return for the long-short portfolio: {:.2f}%'.format(long_short_average_fraction))

Average fraction of winners that outperform the market return for the long-short portfolio: 54.06%


# Answer to Extra credit to Q 4
- The portfolios are necessary in our quant trading strategies because the portfolios allow us to diversify our investments and reduce the risk of investing in a single stock. If we just pick one stock, we are exposed to the risk of that stock. The chance of a single stock beating the market return is not consistent. By investing in a portfolio of stocks, we can reduce the risk of investing in a single stock and have higher overall chance of beating the market return.

# End